In [1]:
from cv2 import cv2 
import numpy as np 
 
left_ear_cascade = cv2.CascadeClassifier('haarcascade_mcs_leftear.xml') 
right_ear_cascade = cv2.CascadeClassifier('haarcascade_mcs_rightear.xml') 
 
if left_ear_cascade.empty(): 
  raise IOError('Unable to load the left ear cascade classifier xml file') 
 
if right_ear_cascade.empty(): 
  raise IOError('Unable to load the right ear cascade classifier xml file') 
 
#cap = cv2.VideoCapture(0)
#scaling_factor = 0.5
image = cv2.imread('er4.jpg') 
print(image.shape)
#print(frame.shape)
#dim = (int(image.shape[1]*0.20),int(image.shape[0]*0.20))
while True:
    #ret, frame = cap.read() 
    frame = cv2.imread('web1.jpg')
    #frame = cv2.resize(frame, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    left_ear = left_ear_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3) 
    right_ear = right_ear_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3) 
    print(frame.shape)
    for (x,y,w,h) in left_ear: 
        
        #cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)
        #cv2.circle(frame,(int(x+w*0.7),int(y+h*0.8)),5,(255,255,255),-1,8,0)
        factor = auto_map_images(frame.shape[:-1],image.shape[:-1],(x,y,x+w,y+h))
        dim = (int(image.shape[1]*factor),int(image.shape[0]*factor))
        img2gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # add a threshold
        ret, mask = cv2.threshold(img2gray, 220, 255, cv2.THRESH_BINARY_INV)
        #cv2.imshow('Ear Detector', ear_img) 
        mask_inv = cv2.bitwise_not(mask)
        resized = cv2.resize(image,dim, interpolation = cv2.INTER_AREA)
        resized1 = cv2.resize(resized, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        #mask = cv2.resize(mask, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        #mask_inv = cv2.resize(mask_inv, (resized.shape[1],resized.shape[0]), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        mask_inv = cv2.resize(mask_inv, (resized.shape[1]*10,resized.shape[0]*10), interpolation = cv2.INTER_AREA)
        print(resized.shape)
        print(mask.shape)
        print(mask_inv.shape)
        x1 = int(x+w*0.7)
        x2 = x1 + dim[1]
        y1 = int(y+h*0.8)
        y2 = y1 + dim[0]
        print(x2-x1,y2-y1)
        print(x,y,x+w,y+h)
        print(w,h)
        if dim[0]%2==1:
            roi1 = frame[int(y+h*0.8):int(y+h*0.8)+dim[1],int(x+w*0.7)-(int(dim[0]/2)):int(x+w*0.7)+(int(dim[0]/2)+1)]
            roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
            print(roi.shape)
            roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
            # roi_fg contains the orignal location of earring
            roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
            # joining the roi_bg and roi_fg
            dst1 = cv2.add(roi_bg,roi_fg)
            dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
            cv2.imshow('Ear Detector', dst) 
            cv2.waitKey(10)
            frame[int(y+h*0.8):int(y+h*0.8)+dim[1],int(x+w*0.7)-(int(dim[0]/2)):int(x+w*0.7)+(int(dim[0]/2)+1)]=dst
        else:
            roi1 = frame[int(y+h*0.8):int(y+h*0.8)+dim[1],int(x+w*0.7)-(int(dim[0]/2)):int(x+w*0.7)+(int(dim[0]/2))]
            roi = cv2.resize(roi1, (roi1.shape[1]*10,roi1.shape[0]*10), interpolation = cv2.INTER_AREA)
            print(roi.shape)
            roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)
            # roi_fg contains the orignal location of earring
            roi_fg = cv2.bitwise_and(resized1,resized1,mask = mask)
            # joining the roi_bg and roi_fg
            dst1 = cv2.add(roi_bg,roi_fg)
            dst = cv2.resize(dst1, (int(dst1.shape[1]/10),int(dst1.shape[0]/10)), interpolation = cv2.INTER_AREA)
            cv2.imshow('Ear Detector', dst) 
            cv2.waitKey(10)
            frame[int(y+h*0.8):int(y+h*0.8)+dim[1],int(x+w*0.7)-(int(dim[0]/2)):int(x+w*0.7)+(int(dim[0]/2))]=dst

    cv2.imshow('Ear Detector', frame)
    #cv2.imshow('Ear Detector', dst)
    
    #c = cv2.waitKey(1) 
    if cv2.waitKey(3000) & 0xFF == ord('q'):
        break

#cap.release() 
cv2.destroyAllWindows()

(174, 113, 3)
(720, 480, 3)
(720, 480, 3)
(720, 480, 3)


In [5]:
def auto_map_images(img_dim,tar_dim,det_dim):
    mul_factor = (det_dim[2]-det_dim[0])/tar_dim[1]
    #print(mul_factor)
    new_tar_dim = tuple([int(mul_factor*x)for x in tar_dim])
    #print(new_tar_dim)
    left_space = img_dim[1]-det_dim[3]
    #print(left_space)
    if left_space>new_tar_dim[0]:
        factor = mul_factor
    else:
        factor = left_space/tar_dim[0]
    return factor